In [172]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from anytree import Node, RenderTree
from pandas import Series


from pandas import read_table
import json

In [173]:
#with open('evaluation_data/R-HSA-1474290/gene_sets.json') as f:
#    data = json.load(f, object_hook=lambda elem: print(elem))
#wv_from_bin = KeyedVectors.load_word2vec_format("__data/nlp/PubMed-Pyysalo-2013/wikipedia-pubmed-and-PMC-w2v.bin", binary=True)  # C binary format
#wv_from_bin["scna"]

#"__data/ppi/BioGrid/BIOGRID-ALL-3.5.166.mitab.txt"

class GeneSet:
    def __init__(self, name, external_id, external_source, summary, entregene_ids, gene_symbols):
        self.name = name
        self.external_id = external_id
        self.external_source = external_source
        self.summary = summary
        self.entregene_ids = entregene_ids
        self.gene_symbols = gene_symbols

    def __repr__(self):
        return "<GeneSet(name='%s', external_id='%s')>" % (self.name, self.external_id)

with open("evaluation_data/immune_cells/gene_sets.json") as fp:
    gene_sets = json.load(fp)

entrezgene2go_df = read_table("annotation_data/entrezgene2go.tsv")
go_df = read_table("annotation_data/go.tsv")

In [177]:




go_anno = entrezgene2go_df.join(go_df.set_index("go_id"), on="go_id")
enrich_go(gene_sets, go_anno)

,entrezgene,go_id,name_1006,definition_1006,go_linkage_type,namespace_1003
93,79042,GO:0003676,nucleic acid binding,Interacting selectively and non-covalently wit...,IEA,molecular_function
93,79042,GO:0003676,nucleic acid binding,Interacting selectively and non-covalently wit...,TAS,molecular_function
93,79042,GO:0003676,nucleic acid binding,Interacting selectively and non-covalently wit...,NAS,molecular_function
93,79042,GO:0003676,nucleic acid binding,Interacting selectively and non-covalently wit...,IBA,molecular_function
93,79042,GO:0003676,nucleic acid binding,Interacting selectively and non-covalently wit...,IMP,molecular_function
93,79042,GO:0003676,nucleic acid binding,Interacting selectively and non-covalently wit...,IDA,molecular_function
94,79042,GO:0006397,mRNA processing,Any process involved in the conversion of a pr...,IEA,biological_process
94,79042,GO:0006397,mRNA processing,Any process involved in the conversion of a pr...,TAS,biological_process
94,79042,GO:0006397,mRNA processing,Any process involved in the conversion of a pr...,IDA,biological_process
94,79042,GO:0006397,mRNA processing,Any process involved in the conversion of a pr...,IMP,biological_process
